In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings(action="ignore")

In [ ]:
df_dirty_train = pd.read_csv('../data/train.csv')
df_dirty_test = pd.read_csv('../data/test.csv')
df_dirty_train.head()

In [ ]:
#a.	Remove duplicates

In [ ]:
#b.	Remove irrelevant data

In [ ]:
#c.	Standardize capitalization

In [ ]:
#d.	Convert data type

In [ ]:
#e.	Clear formatting

In [ ]:
#f.	Fix errors

In [ ]:
#g.	Language translation

In [ ]:
#h.	Handle missing values

In [ ]:
#No need to run this unless all data has been cleaned.
df_dirty_test.to_csv('../data/test_cleaned.csv')  
df_dirty_train.to_csv('../data/train_cleaned.csv')